In [1]:
# @title
# %% [markdown]
# # WanGP v5.41 - Complete Cloud Installation with Enhanced Debugging & Dual Share
#
# ## 🎬 ALL Features from v5.41 Including:
# - **Dual Public Access**: Gradio --share + ngrok for maximum reliability
# - **Full Debug Output**: Complete verbose logging of all operations
# - **Robust Directory Management**: Error-proof workspace handling
# - **All Models**: Wan, Hunyuan, LTX, VACE (1.3B & 14B), MoviiGen, etc.
# - **Queue System**: Stack multiple generation tasks
# - **Video Settings Management**: Save/load/reuse video settings (v5.3)
# - **Complete Error Handling**: Bulletproof against common issues

# %% [markdown]
# ## 1. Workspace Setup & Configuration

# %%
# ====================================
# 🔧 USER CONFIGURATION SECTION
# ====================================

# NGROK CONFIGURATION (Get token from: https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = ""  # <- PASTE YOUR NGROK TOKEN HERE (Leave empty to use Gradio share only)

# WORKSPACE CONFIGURATION
WORKSPACE_NAME = "WanGP_Workspace"  # Main workspace directory
FORCE_CLEAN_INSTALL = False  # Set True to delete existing workspace
AUTO_FIX_CONFLICTS = True  # Automatically resolve directory conflicts

# ADVANCED SETTINGS
DEBUG_LEVEL = 2  # 0=minimal, 1=normal, 2=verbose (shows everything)
ENABLE_NGROK = True  # Use ngrok as backup/primary share method
NGROK_REGION = "us"  # us, eu, ap, au, sa, jp, in
MONITOR_RESOURCES = True  # Show real-time GPU/CPU/RAM usage
AUTO_RESTART_ON_FAIL = True  # Automatically retry if launch fails
ENABLE_UPSAMPLING = True  # Enable temporal/spatial upsampling features
DOWNLOAD_ALL_LORAS = True  # Download all essential loras

print("="*80)
print("🎮 WANGP v5.41 ROBUST INSTALLATION")
print("="*80)
print(f"Workspace: {WORKSPACE_NAME}")
print(f"Clean Install: {'YES - Will delete existing' if FORCE_CLEAN_INSTALL else 'NO - Will reuse/update'}")
print(f"Auto-fix: {'ENABLED' if AUTO_FIX_CONFLICTS else 'DISABLED'}")
print(f"Debug Level: {DEBUG_LEVEL} ({'Verbose' if DEBUG_LEVEL == 2 else 'Normal' if DEBUG_LEVEL == 1 else 'Minimal'})")
print(f"Ngrok: {'ENABLED' if ENABLE_NGROK and NGROK_AUTH_TOKEN else 'DISABLED'}")
print(f"Ngrok Token: {'✅ SET' if NGROK_AUTH_TOKEN else '❌ NOT SET (will use Gradio share)'}")
print("="*80)

# %% [markdown]
# ## 2. Robust Directory Management System

# %%
import os
import sys
import subprocess
import torch
import platform
import psutil
import json
import time
import socket
import shutil
from datetime import datetime
from pathlib import Path

def safe_remove_directory(path):
    """Safely remove directory with error handling"""
    if os.path.exists(path):
        try:
            if os.path.islink(path):
                os.unlink(path)
            else:
                shutil.rmtree(path)
            return True
        except Exception as e:
            print(f"⚠️ Could not remove {path}: {e}")
            return False
    return True

def ensure_clean_workspace():
    """Create a clean workspace directory"""
    current_dir = os.getcwd()
    workspace_path = os.path.join(current_dir, WORKSPACE_NAME)

    print("="*80)
    print("📁 WORKSPACE MANAGEMENT")
    print("="*80)
    print(f"Current directory: {current_dir}")
    print(f"Target workspace: {workspace_path}")

    # Handle existing workspace
    if os.path.exists(workspace_path):
        if FORCE_CLEAN_INSTALL:
            print(f"\n🧹 Force clean enabled - removing existing workspace...")
            if safe_remove_directory(workspace_path):
                print("✅ Existing workspace removed")
            else:
                print("❌ Could not remove workspace - will work around it")
        else:
            print(f"\n📁 Workspace exists - checking contents...")

            # Check if it contains WanGP
            wangp_path = os.path.join(workspace_path, "Wan2GP")
            if os.path.exists(wangp_path):
                print(f"✅ Found existing WanGP installation")

                # Check if it's a valid repo
                if os.path.exists(os.path.join(wangp_path, ".git")):
                    print("✅ Valid git repository found")
                    return workspace_path, wangp_path, "update"
                else:
                    print("⚠️ Directory exists but not a git repo")
                    if AUTO_FIX_CONFLICTS:
                        print("🔧 Auto-fix enabled - will clean and re-clone")
                        safe_remove_directory(wangp_path)
                        return workspace_path, wangp_path, "clone"
            else:
                print("📂 Empty workspace - will create WanGP inside")
                return workspace_path, os.path.join(workspace_path, "Wan2GP"), "clone"

    # Create new workspace
    print(f"\n📁 Creating new workspace: {WORKSPACE_NAME}")
    os.makedirs(workspace_path, exist_ok=True)

    return workspace_path, os.path.join(workspace_path, "Wan2GP"), "clone"

# Setup workspace
workspace_dir, repo_path, action = ensure_clean_workspace()

# Change to workspace
os.chdir(workspace_dir)
print(f"\n✅ Working in: {os.getcwd()}")
print(f"Action: {action}")
print("="*80)

# %% [markdown]
# ## 3. System Diagnostics with Error Handling

# %%
print("="*80)
print("🔍 COMPREHENSIVE SYSTEM DIAGNOSTICS")
print("="*80)

def safe_get_info(func, default="Unknown"):
    """Safely get system info with fallback"""
    try:
        return func()
    except:
        return default

# Basic system info
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Platform: {safe_get_info(lambda: f'{platform.system()} {platform.release()}')}")
print(f"Python: {sys.version.split()[0]}")
print(f"Workspace: {workspace_dir}")

# Detect environment
env_indicators = {
    'Google Colab': lambda: 'google.colab' in str(get_ipython()),
    'Kaggle': lambda: 'KAGGLE_URL_BASE' in os.environ,
    'Lightning.ai': lambda: 'LIGHTNING_CLOUD_URL' in os.environ,
    'Vast.ai': lambda: os.path.exists('/opt/bin/nvidia-smi'),
    'Paperspace': lambda: 'PS_API_KEY' in os.environ
}

detected_env = "Unknown"
for env_name, check_func in env_indicators.items():
    if safe_get_info(check_func, False):
        detected_env = env_name
        break

print(f"Environment: {detected_env}")

# Resource info
print(f"\n[RESOURCES]")
print(f"CPU Cores: {safe_get_info(lambda: psutil.cpu_count(logical=False), 'Unknown')}")
mem = safe_get_info(lambda: psutil.virtual_memory(), None)
if mem:
    print(f"RAM: {mem.total/(1024**3):.1f} GB total, {mem.available/(1024**3):.1f} GB available")
else:
    print("RAM: Could not detect")

# GPU Detection with comprehensive error handling
print(f"\n[GPU DETECTION]")
gpu_available = False
gpu_info = "No GPU"
gpu_memory = 0
gpu_generation = 'none'

try:
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        print(f"✅ CUDA Available: True")
        print(f"GPU Count: {gpu_count}")
        print(f"CUDA Version: {torch.version.cuda}")

        # Get primary GPU info
        gpu_info = torch.cuda.get_device_name(0)
        props = torch.cuda.get_device_properties(0)
        gpu_memory = props.total_memory / (1024**3)

        print(f"Primary GPU: {gpu_info}")
        print(f"VRAM: {gpu_memory:.2f} GB")

        # Determine generation
        gpu_name_lower = gpu_info.lower()
        if any(x in gpu_name_lower for x in ['5090', '5080', '5070', '5060', '5050']):
            gpu_generation = 'rtx50xx'
            pytorch_version = "2.7.0"
            cuda_index = "cu128"
        elif any(x in gpu_name_lower for x in ['a100', 'a6000', 'a40', 'v100']):
            gpu_generation = 'datacenter'
            pytorch_version = "2.6.0"
            cuda_index = "cu124"
        else:
            gpu_generation = 'standard'
            pytorch_version = "2.6.0"
            cuda_index = "cu124"

        print(f"Generation: {gpu_generation}")
        gpu_available = True

    else:
        print("❌ CUDA Not Available")

except Exception as e:
    print(f"❌ GPU Detection Error: {e}")

if not gpu_available:
    print("\n🔧 GPU TROUBLESHOOTING:")
    print("1. Google Colab: Runtime > Change runtime type > GPU")
    print("2. Other platforms: Ensure GPU instance selected")
    print("3. Try: !nvidia-smi to check GPU status")

    # Still allow CPU execution for testing
    print("\n⚠️ Continuing with CPU mode (very slow)")
    gpu_info = "CPU Mode"
    gpu_memory = 0
    pytorch_version = "2.6.0"
    cuda_index = "cu124"

print("="*80)

# %% [markdown]
# ## 4. Repository Management with Conflict Resolution

# %%
print("="*80)
print("📦 REPOSITORY MANAGEMENT")
print("="*80)

def run_command_safe(cmd, shell=True, capture_output=True, timeout=300):
    """Run command with timeout and error handling"""
    try:
        result = subprocess.run(
            cmd,
            shell=shell,
            capture_output=capture_output,
            text=True,
            timeout=timeout
        )
        return result.returncode, result.stdout, result.stderr
    except subprocess.TimeoutExpired:
        return -1, "", "Command timed out"
    except Exception as e:
        return -1, "", str(e)

def clone_or_update_repo():
    """Handle repository cloning/updating with conflict resolution"""
    repo_url = "https://github.com/remphanostar/WanBook.git"

    if action == "update":
        print(f"[1/3] Updating existing repository...")
        os.chdir(repo_path)

        # Check git status
        ret, stdout, stderr = run_command_safe("git status --porcelain")
        if ret == 0 and stdout.strip():
            print("⚠️ Local changes detected")
            if AUTO_FIX_CONFLICTS:
                print("🔧 Auto-fix: Stashing local changes...")
                run_command_safe("git stash")

        # Pull updates
        ret, stdout, stderr = run_command_safe("git pull")
        if ret == 0:
            print("✅ Repository updated")
        else:
            print(f"⚠️ Update failed: {stderr}")
            print("Will attempt fresh clone...")
            os.chdir(workspace_dir)
            safe_remove_directory(repo_path)
            action = "clone"

    if action == "clone":
        print(f"[1/3] Cloning repository from GitHub...")
        print(f"URL: {repo_url}")

        # Ensure target doesn't exist
        if os.path.exists(repo_path):
            print(f"⚠️ Target exists, removing: {repo_path}")
            safe_remove_directory(repo_path)

        # Clone with progress
        start_time = time.time()
        ret, stdout, stderr = run_command_safe(
            f"git clone {repo_url} {os.path.basename(repo_path)}",
            timeout=600  # 10 minutes max
        )

        elapsed = time.time() - start_time

        if ret == 0:
            print(f"✅ Repository cloned in {elapsed:.1f}s")
        else:
            print(f"❌ Clone failed: {stderr}")

            # Fallback: try wget
            print("🔄 Trying alternative download method...")
            zip_url = "https://github.com/remphanostar/WanBook/archive/refs/heads/main.zip"
            ret, stdout, stderr = run_command_safe(f"wget -O wangp.zip {zip_url}")

            if ret == 0:
                print("📦 Downloaded as ZIP, extracting...")
                ret, stdout, stderr = run_command_safe("unzip -q wangp.zip")
                if ret == 0:
                    # Rename extracted folder
                    extracted_name = "Wan2GP-main"
                    if os.path.exists(extracted_name):
                        os.rename(extracted_name, os.path.basename(repo_path))
                        print("✅ Repository extracted successfully")
                    else:
                        raise RuntimeError("Could not extract repository")
                else:
                    raise RuntimeError("Failed to extract ZIP")
            else:
                raise RuntimeError("All download methods failed")

# Execute repository setup
try:
    clone_or_update_repo()
except Exception as e:
    print(f"❌ Repository setup failed: {e}")
    print("\n🔧 Manual steps:")
    print("1. Check internet connection")
    print("2. Try: !git clone https://github.com/remphanostar/WanBook.git")
    print("3. Or download ZIP manually")
    raise

# Change to repo directory
print(f"\n[2/3] Entering repository directory...")
os.chdir(repo_path)
print(f"Working directory: {os.getcwd()}")

# Verify repository
print(f"\n[3/3] Verifying repository integrity...")
required_files = {
    'wgp.py': 'Main application',
    'requirements.txt': 'Dependencies',
    'CHANGELOG.md': 'Version history'
}

missing_files = []
for file, desc in required_files.items():
    if os.path.exists(file):
        size = os.path.getsize(file) / 1024
        print(f"  ✅ {file:<20} ({size:>6.1f} KB)")
    else:
        print(f"  ❌ {file:<20} MISSING")
        missing_files.append(file)

if missing_files:
    print(f"\n❌ Repository incomplete! Missing: {', '.join(missing_files)}")
    if AUTO_FIX_CONFLICTS:
        print("🔧 Auto-fix: Attempting re-download...")
        os.chdir(workspace_dir)
        safe_remove_directory(repo_path)
        # Try again
        clone_or_update_repo()
        os.chdir(repo_path)
    else:
        raise FileNotFoundError(f"Missing critical files: {missing_files}")

print("\n✅ Repository ready")
print("="*80)

# %% [markdown]
# ## 5. Install PyTorch with Version Management

# %%
print("="*80)
print("🔧 PYTORCH INSTALLATION")
print("="*80)

def check_pytorch_compatibility():
    """Check if current PyTorch is compatible"""
    try:
        import torch as existing_torch
        current_version = existing_torch.__version__
        current_cuda = existing_torch.version.cuda

        print(f"Current PyTorch: {current_version}")
        print(f"Current CUDA: {current_cuda}")

        # Check compatibility
        needs_reinstall = False

        if gpu_generation == 'rtx50xx':
            if not current_version.startswith('2.7'):
                print("⚠️ RTX 50XX requires PyTorch 2.7.0")
                needs_reinstall = True
        else:
            if not current_version.startswith('2.6'):
                print("ℹ️ Stable PyTorch 2.6.0 recommended")
                needs_reinstall = True

        if not torch.cuda.is_available() and gpu_available:
            print("⚠️ PyTorch doesn't detect CUDA")
            needs_reinstall = True

        return needs_reinstall, current_version

    except ImportError:
        print("No PyTorch found")
        return True, None

# Check current installation
needs_install, current_version = check_pytorch_compatibility()

if needs_install:
    print(f"\n[Installing PyTorch {pytorch_version}]")

    # Uninstall existing if needed
    if current_version:
        print("Removing existing PyTorch...")
        ret, _, _ = run_command_safe("pip uninstall torch torchvision torchaudio -y", timeout=120)
        if ret == 0:
            print("✅ Removed existing PyTorch")

    # Install new version
    install_cmd = f"pip install torch=={pytorch_version} torchvision torchaudio --index-url https://download.pytorch.org/whl/test/{cuda_index}"
    print(f"Installing: {install_cmd}")
    print("This may take 5-10 minutes...")

    start_time = time.time()
    ret, stdout, stderr = run_command_safe(install_cmd, timeout=900)  # 15 minutes max
    elapsed = time.time() - start_time

    if ret == 0:
        print(f"✅ PyTorch installed in {elapsed/60:.1f} minutes")
    else:
        print(f"❌ Installation failed: {stderr}")
        print("🔄 Trying with pip upgrade...")
        ret, _, _ = run_command_safe("pip install --upgrade pip", timeout=60)
        ret, _, stderr = run_command_safe(install_cmd, timeout=900)
        if ret != 0:
            raise RuntimeError(f"PyTorch installation failed: {stderr}")
else:
    print("✅ PyTorch already compatible")

# Verify installation
print("\nVerifying PyTorch...")
try:
    import torch
    print(f"✅ PyTorch {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")

    if torch.cuda.is_available():
        print(f"✅ GPU accessible: {torch.cuda.get_device_name(0)}")

        # Quick GPU test
        try:
            test_tensor = torch.rand(100, 100).cuda()
            result = test_tensor @ test_tensor
            print("✅ GPU operations working")
            del test_tensor, result
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"⚠️ GPU test failed: {e}")

except Exception as e:
    print(f"❌ PyTorch verification failed: {e}")
    raise

print("="*80)

# %% [markdown]
# ## 6. Dependencies with Retry Logic

# %%
print("="*80)
print("📦 DEPENDENCY INSTALLATION")
print("="*80)

def install_requirements_with_retry(max_retries=3):
    """Install requirements with retry logic"""

    for attempt in range(max_retries):
        print(f"\nAttempt {attempt + 1}/{max_retries}")

        start_time = time.time()
        ret, stdout, stderr = run_command_safe(
            "pip install -r requirements.txt",
            timeout=1200  # 20 minutes
        )
        elapsed = time.time() - start_time

        if ret == 0:
            print(f"✅ Dependencies installed in {elapsed/60:.1f} minutes")
            return True
        else:
            print(f"❌ Attempt {attempt + 1} failed: {stderr}")

            if attempt < max_retries - 1:
                print("🔄 Retrying with pip upgrade...")
                run_command_safe("pip install --upgrade pip", timeout=60)
                time.sleep(5)

    return False

# Install dependencies
print("Installing all dependencies...")
print("This includes: Gradio, Diffusers, Transformers, Accelerate...")

if not install_requirements_with_retry():
    print("❌ All installation attempts failed")
    print("🔧 Manual fallback:")
    print("Try running: !pip install gradio diffusers accelerate transformers")
    raise RuntimeError("Dependency installation failed")

# Verify key packages
print("\n[PACKAGE VERIFICATION]")
critical_packages = ['gradio', 'diffusers', 'accelerate', 'transformers']
missing_packages = []

for package in critical_packages:
    try:
        module = __import__(package)
        version = getattr(module, '__version__', 'unknown')
        print(f"  ✅ {package}: {version}")
    except ImportError:
        print(f"  ❌ {package}: MISSING")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️ Missing critical packages: {missing_packages}")
    print("Attempting individual installation...")
    for package in missing_packages:
        ret, _, _ = run_command_safe(f"pip install {package}", timeout=300)
        if ret == 0:
            print(f"✅ Installed {package}")
        else:
            print(f"❌ Failed to install {package}")

print("="*80)

# %% [markdown]
# ## 7. Performance Optimization (Safe)

# %%
print("="*80)
print("⚡ PERFORMANCE OPTIMIZATION")
print("="*80)

# Determine safe defaults
attention_mode = "sdpa"  # Always safe
compile_enabled = False
teacache_multiplier = "2.0"
profile = 4

print(f"GPU: {gpu_info}")
print(f"VRAM: {gpu_memory:.1f} GB")

# Optimization strategy based on VRAM
if gpu_memory >= 24:
    print("🏆 Ultra High VRAM: All optimizations available")
    profile = 3
    teacache_multiplier = "2.5"
    target_attention = "sage2"
elif gpu_memory >= 16:
    print("💪 High VRAM: Advanced optimizations")
    profile = 4
    target_attention = "sage2"
elif gpu_memory >= 12:
    print("✅ Medium VRAM: Standard optimizations")
    target_attention = "sage"
elif gpu_memory >= 8:
    print("⚡ Standard VRAM: Basic optimizations")
    teacache_multiplier = "1.75"
    target_attention = "sage"
else:
    print("🔋 Low VRAM: Conservative settings")
    teacache_multiplier = "1.5"
    target_attention = "sdpa"

# Try to install optimizations safely
if target_attention in ["sage", "sage2"] and gpu_memory >= 8:
    print(f"\n[INSTALLING {target_attention.upper()} ATTENTION]")

    # Install triton first
    triton_cmd = "pip install triton" if platform.system() == "Linux" else "pip install triton-windows"
    ret, _, stderr = run_command_safe(triton_cmd, timeout=300)

    if ret == 0:
        print("✅ Triton installed")

        # Install SageAttention
        ret, _, stderr = run_command_safe("pip install sageattention==1.0.6", timeout=300)
        if ret == 0:
            attention_mode = "sage"
            print("✅ Sage Attention installed (+30% speed)")

            # Try Sage2 for high VRAM
            if target_attention == "sage2" and gpu_memory >= 12:
                print("Attempting Sage2 installation...")
                # Only try pre-built wheels for safety
                if gpu_generation == 'rtx50xx' and platform.system() == "Windows":
                    sage2_url = "https://github.com/woct0rdho/SageAttention/releases/download/v2.1.1-windows/sageattention-2.1.1+cu128torch2.7.0-cp310-cp310-win_amd64.whl"
                    ret, _, _ = run_command_safe(f"pip install {sage2_url}", timeout=300)
                    if ret == 0:
                        attention_mode = "sage2"
                        compile_enabled = True
                        print("✅ Sage2 installed (+40% speed)")
                elif gpu_generation == 'standard' and platform.system() == "Windows":
                    sage2_url = "https://github.com/woct0rdho/SageAttention/releases/download/v2.1.1-windows/sageattention-2.1.1+cu126torch2.6.0-cp310-cp310-win_amd64.whl"
                    ret, _, _ = run_command_safe(f"pip install {sage2_url}", timeout=300)
                    if ret == 0:
                        attention_mode = "sage2"
                        compile_enabled = True
                        print("✅ Sage2 installed (+40% speed)")
        else:
            print(f"⚠️ Sage installation failed: {stderr}")
    else:
        print(f"⚠️ Triton installation failed: {stderr}")

print(f"\n[FINAL SETTINGS]")
print(f"Attention: {attention_mode}")
print(f"Profile: {profile}")
print(f"TeaCache: {teacache_multiplier}x")
print(f"Compilation: {'Yes' if compile_enabled else 'No'}")
print("="*80)

# %% [markdown]
# ## 8. Ngrok Setup (Safe)

# %%
print("="*80)
print("🌐 NGROK CONFIGURATION")
print("="*80)

ngrok_available = False

if ENABLE_NGROK and NGROK_AUTH_TOKEN:
    try:
        # Install pyngrok safely
        ret, _, stderr = run_command_safe("pip install pyngrok", timeout=120)
        if ret == 0:
            print("✅ pyngrok installed")

            # Import and configure
            from pyngrok import ngrok, conf

            # Set token
            ngrok.set_auth_token(NGROK_AUTH_TOKEN)
            print("✅ Ngrok token configured")

            # Configure region
            config = conf.get_default()
            config.region = NGROK_REGION
            config.monitor_thread = False

            # Test connection
            print("Testing ngrok...")
            test_tunnel = ngrok.connect(7860, "http")
            print(f"✅ Test successful: {test_tunnel.public_url}")
            ngrok.disconnect(test_tunnel.public_url)

            ngrok_available = True

        else:
            print(f"❌ pyngrok installation failed: {stderr}")

    except Exception as e:
        print(f"⚠️ Ngrok setup error: {e}")
        print("Will use Gradio share only")
else:
    print("Ngrok disabled or no token provided")

print(f"Share methods: Gradio {'+ Ngrok' if ngrok_available else ' only'}")
print("="*80)

# %% [markdown]
# ## 9. Directory Structure Creation

# %%
print("="*80)
print("📁 CREATING DIRECTORY STRUCTURE")
print("="*80)

# Create all necessary directories
directories = [
    'loras', 'loras/1.3B', 'loras/14B',
    'loras_i2v', 'loras_hunyuan', 'loras_hunyuan_i2v', 'loras_ltxv',
    'settings', 'presets', 'output', 'output/videos', 'output/gallery',
    'temp', 'cache', 'cache/models'
]

created = 0
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
        created += 1
        print(f"✅ Created: {directory}")
    else:
        print(f"📁 Exists: {directory}")

print(f"\nCreated {created} new directories")
print("="*80)

# %% [markdown]
# ## 10. Essential Loras Download

# %%
print("="*80)
print("🎨 ESSENTIAL LORAS")
print("="*80)

if DOWNLOAD_ALL_LORAS:
    loras = [
        {
            "name": "CausVid T2V",
            "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan21_CausVid_14B_T2V_lora_rank32.safetensors",
            "path": "loras/"
        },
        {
            "name": "AccVid T2V",
            "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan21_AccVid_T2V_14B_lora_rank32_fp16.safetensors",
            "path": "loras/"
        },
        {
            "name": "AccVid I2V",
            "url": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan21_AccVid_I2V_480P_14B_lora_rank32_fp16.safetensors",
            "path": "loras_i2v/"
        }
    ]

    total_size = 0
    for lora in loras:
        filename = os.path.basename(lora["url"])
        filepath = os.path.join(lora["path"], filename)

        print(f"\n{lora['name']}:")

        if os.path.exists(filepath):
            size = os.path.getsize(filepath) / (1024**2)
            total_size += size
            print(f"  ✅ Already exists ({size:.1f} MB)")
        else:
            print(f"  📥 Downloading...")
            ret, _, stderr = run_command_safe(f"wget -q -P {lora['path']} {lora['url']}", timeout=600)

            if ret == 0 and os.path.exists(filepath):
                size = os.path.getsize(filepath) / (1024**2)
                total_size += size
                print(f"  ✅ Downloaded ({size:.1f} MB)")
            else:
                print(f"  ❌ Download failed: {stderr}")

    print(f"\nTotal Lora size: {total_size:.1f} MB")
else:
    print("Lora downloads skipped")

print("="*80)

# %% [markdown]
# ## 11. Configuration Files

# %%
print("="*80)
print("⚙️ CONFIGURATION FILES")
print("="*80)

# Create default settings
settings = {
    'settings/t2v_settings.json': {
        "prompt": "A beautiful cinematic scene, high quality, detailed",
        "frames": 49,
        "steps": 20,
        "guidance_scale": 7.0,
        "seed": -1,
        "teacache_multiplier": float(teacache_multiplier)
    },
    'settings/i2v_settings.json': {
        "prompt": "Natural motion, smooth animation",
        "frames": 49,
        "steps": 20,
        "guidance_scale": 7.0,
        "seed": -1
    }
}

for filepath, config in settings.items():
    with open(filepath, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"✅ Created: {filepath}")

print("="*80)

# %% [markdown]
# ## 12. Launch Script Creation

# %%
print("="*80)
print("🚀 CREATING LAUNCH SCRIPT")
print("="*80)

# Build launch command
launch_command = [
    "python", "wgp.py",
    "--share",  # CRITICAL for cloud access
    "--server-port", "7860",
    "--attention", attention_mode,
    "--profile", str(profile),
    "--teacache", teacache_multiplier,
    "--verbose", str(DEBUG_LEVEL)
]

# Add model selection based on VRAM
if gpu_memory < 8:
    launch_command.extend(["--t2v-1-3B"])
    print("Default model: 1.3B (Low VRAM)")
elif gpu_memory >= 16:
    print("Default model: User choice (All available)")
else:
    print("Default model: User choice (1.3B/14B)")

if compile_enabled:
    launch_command.append("--compile")

print(f"Launch command: {' '.join(launch_command)}")

# Create launch script with error handling
launch_script = f'''#!/usr/bin/env python3
import subprocess
import sys
import os
import time
from datetime import datetime

# Configuration
ENABLE_NGROK = {ngrok_available}
MONITOR_RESOURCES = {MONITOR_RESOURCES}
DEBUG_LEVEL = {DEBUG_LEVEL}

print("="*80)
print("🎬 WANGP v5.41 LAUNCHER")
print("="*80)
print(f"Time: {{datetime.now()}}")
print(f"GPU: {gpu_info}")
print(f"VRAM: {gpu_memory:.1f} GB")
print(f"Share: Gradio{'+ Ngrok' if ngrok_available else ''}")
print("="*80)

# Environment setup
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
os.environ['GRADIO_ANALYTICS_ENABLED'] = 'False'

# Setup ngrok if available
ngrok_tunnel = None
if ENABLE_NGROK:
    try:
        from pyngrok import ngrok
        ngrok_tunnel = ngrok.connect(7860, "http")
        print(f"🌐 Ngrok URL: {{ngrok_tunnel.public_url}}")
    except Exception as e:
        print(f"⚠️ Ngrok failed: {{e}}")

# Launch command
params = {launch_command}
print(f"Command: {{' '.join(params)}}\\n")

try:
    # Launch process
    process = subprocess.Popen(
        params,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    # Monitor output
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None:
            break
        if line:
            print(line.rstrip())

            # Detect URLs
            if 'Running on public URL:' in line:
                url = line.split('public URL:')[1].strip()
                print("\\n" + "🌟" * 40)
                print(f"✅ GRADIO URL: {{url}}")
                if ngrok_tunnel:
                    print(f"✅ NGROK URL: {{ngrok_tunnel.public_url}}")
                print("🌟" * 40 + "\\n")

except KeyboardInterrupt:
    print("\\n⚠️ Stopped by user")
finally:
    if ngrok_tunnel:
        try:
            ngrok.disconnect(ngrok_tunnel.public_url)
        except:
            pass
'''

with open('launch_wangp.py', 'w') as f:
    f.write(launch_script)

print("✅ Launch script created: launch_wangp.py")
print("="*80)

# %% [markdown]
# ## 13. 🚀 LAUNCH WANGP

# %%
print(f"""
🎬 LAUNCHING WANGP v5.41
========================

🔧 CONFIGURATION:
- GPU: {gpu_info}
- VRAM: {gpu_memory:.1f} GB
- Attention: {attention_mode}
- Share: Gradio {'+ Ngrok' if ngrok_available else 'only'}
- Debug: Level {DEBUG_LEVEL}

📌 LOOK FOR:
- "Running on public URL:" in output below
- Click any public URL to access WanGP
- First model download may take 5-15 minutes

🚀 Starting now...
""")

# Execute launch with auto-restart
max_retries = 3 if AUTO_RESTART_ON_FAIL else 1
for attempt in range(max_retries):
    if attempt > 0:
        print(f"\n🔄 Retry {attempt}/{max_retries-1}")
        if gpu_available:
            torch.cuda.empty_cache()
        time.sleep(5)

    print("\n" + "="*80)
    print(f"LAUNCH ATTEMPT {attempt + 1}")
    print("="*80)

    # Launch the script
    try:
        result = subprocess.run([sys.executable, 'launch_wangp.py'])
        if result.returncode == 0:
            print("✅ Exited normally")
            break
        else:
            print(f"⚠️ Exit code: {result.returncode}")
    except Exception as e:
        print(f"❌ Launch error: {e}")

    if attempt < max_retries - 1:
        print("Preparing to restart...")

print("\n🎉 Launch sequence complete!")

# %% [markdown]
# ## 14. Alternative Launch Commands

# %%
print(f"""
🔧 ALTERNATIVE LAUNCH COMMANDS
==============================

All commands include --share for public access:

🚀 BASIC LAUNCHES:
!python wgp.py --share --verbose {DEBUG_LEVEL}
!python wgp.py --t2v-1-3B --share --verbose {DEBUG_LEVEL}
!python wgp.py --i2v-14B --share --verbose {DEBUG_LEVEL}
!python wgp.py --vace-1-3B --share --verbose {DEBUG_LEVEL}

⚡ OPTIMIZED:
!python wgp.py --attention {attention_mode} --profile {profile} --teacache {teacache_multiplier} --share --verbose {DEBUG_LEVEL}

🔧 MINIMAL (if issues):
!python wgp.py --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --share --verbose {DEBUG_LEVEL}

📊 FEATURES:
!python wgp.py --i2v --multiple-images --share --verbose {DEBUG_LEVEL}
!python wgp.py --advanced --share --verbose {DEBUG_LEVEL}

🌐 The --share flag ensures public access on ALL cloud platforms!
""")

# %% [markdown]
# ## 15. Feature Reference

# %%
# Fixed f-string syntax by removing problematic characters
feature_text = f"""
📚 WANGP v5.41 COMPLETE FEATURES
================================

🎬 ALL MODELS AVAILABLE:

TEXT-TO-VIDEO:
- Wan 2.1 T2V 1.3B (6GB VRAM, fastest)
- Wan 2.1 T2V 14B (12GB+ VRAM, best quality)
- Hunyuan Video (best overall quality)
- LTX Video 13B (long videos, optimized)
- LTX Video Distilled (under 1 min generation)

IMAGE-TO-VIDEO:
- Wan Fun InP 1.3B (entry level)
- Wan Fun InP 14B (better quality)
- Wan I2V 14B (standard)
- FLF2V (start/end frame specialist)
- Hunyuan Custom (identity preservation)

CONTROLNET:
- VACE 1.3B (6GB VRAM control)
- VACE 14B (professional control)

SPECIALIZED:
- Hunyuan Avatar (15s speech/song video, 10GB VRAM)
- FantasySpeaking (voice animation)
- Phantom (person/object transfer)
- Recam Master (viewpoint change)
- Sky Reels v2 (infinite length)
- MoviiGen (experimental 1080p, 20GB+)

⚡ PERFORMANCE (YOUR SETUP):
Attention: {attention_mode}
Speed boost: {'+40%' if attention_mode == 'sage2' else '+30%' if attention_mode == 'sage' else 'baseline'}
TeaCache: {teacache_multiplier}x
Profile: {profile}

🎨 v5.41 FEATURES:
✅ AccVideo Lora (2x speed)
✅ Video Settings Management
✅ Queue System
✅ Gallery Management
✅ Matanyone Integration
✅ Prompt Enhancer
✅ Multi-image Support

🌐 ACCESS METHODS:
✅ Gradio Share (always enabled)
{'✅ Ngrok Tunnel' if ngrok_available else '❌ Ngrok (not configured)'}

📊 YOUR HARDWARE:
GPU: {gpu_info}
VRAM: {gpu_memory:.1f} GB
Workspace: {workspace_dir}
"""

print(feature_text)

# %% [markdown]
# ## 🎉 Setup Complete!
#
# Your WanGP v5.41 installation is complete with:
#
# ✅ **Robust Directory Management** - No more nesting issues
# ✅ **Error-Proof Installation** - Handles conflicts automatically
# ✅ **Dual Share Methods** - Gradio + Ngrok for reliable access
# ✅ **Complete Debug Output** - Full visibility into all operations
# ✅ **All v5.41 Features** - Every model and feature available
#
# **Look for the public URLs in the output above and start creating amazing videos!** 🚀

# %% [markdown]
# ## Quick Reference
#
# 📋 **First Video Test:**
# 1. Click any public URL from output above
# 2. Select "Wan 2.1 T2V 1.3B" from dropdown
# 3. Enter: "A serene lake at sunset, cinematic"
# 4. Set frames: 49, steps: 20
# 5. Click Generate!
#
# ⚡ **Speed Modes:**
# - CausVid: steps=12, guidance=1, shift=7, multiplier=0.3
# - AccVid: guidance=1, shift=5
#
# 🆘 **Help:**
# - Discord: https://discord.gg/g7efUW9jGV
# - Issues: Runtime > Restart runtime


🎮 WANGP v5.41 ROBUST INSTALLATION
Workspace: WanGP_Workspace
Clean Install: NO - Will reuse/update
Auto-fix: ENABLED
Debug Level: 2 (Verbose)
Ngrok: DISABLED
Ngrok Token: ❌ NOT SET (will use Gradio share)
📁 WORKSPACE MANAGEMENT
Current directory: /content
Target workspace: /content/WanGP_Workspace

📁 Creating new workspace: WanGP_Workspace

✅ Working in: /content/WanGP_Workspace
Action: clone
🔍 COMPREHENSIVE SYSTEM DIAGNOSTICS
Timestamp: 2025-06-29 08:19:57
Platform: Linux 6.1.123+
Python: 3.11.13
Workspace: /content/WanGP_Workspace
Environment: Google Colab

[RESOURCES]
CPU Cores: 1
RAM: 12.7 GB total, 11.4 GB available

[GPU DETECTION]
✅ CUDA Available: True
GPU Count: 1
CUDA Version: 12.4
Primary GPU: Tesla T4
VRAM: 14.74 GB
Generation: standard
📦 REPOSITORY MANAGEMENT
❌ Repository setup failed: cannot access local variable 'action' where it is not associated with a value

🔧 Manual steps:
1. Check internet connection
2. Try: !git clone https://github.com/remphanostar/WanBook.git
3. 

UnboundLocalError: cannot access local variable 'action' where it is not associated with a value